In [3]:
from collections import  defaultdict
import mysql.connector
import datetime
import json
query=("select my_ip,next_ip,next_port,prev_circ_id,next_circ_id,direction,stream_id,is_origin from net "
      "where time between %s and %s ")
config={}
with open("config") as f:
    t=f.readlines()
    config=json.loads(t)
    

# id,my_ip,next_ip,next_port,prev_circ_id,next_circ_id,direction,stream_id,is_origin
class link:
    def __init__(self,my_ip,next_ip,next_port,prev_circ_id,next_circ_id,direction,stream_id=0,is_origin=False,id=0):
        self.prev_circ_id=prev_circ_id
        self.next_circ_id=next_circ_id
        self.id=id
        self.my_ip=my_ip
        self.next_ip=next_ip
        self.next_port=next_port
        self.direction=direction
        self.stream_id=stream_id
        self.is_origin=is_origin

def readFromDB(offset=30):
    '''
    using my_ip as key
    using prev_circ as second key (accelerate search whether prev_circ is there)
    
    type offset:int  (minute)
    rtype dict() --{"my_ip":{"prev_circ_id":link}}
    '''
    relay=defaultdict(dict)
    origin=defaultdict(dict)
    
    time_start="(select date_sub( ( select now() ), interval %d minute ))"%offset
    time_end="( select now() )"

    cnx = mysql.connector.connect(**config)
    cursor=cnx.cursor()
    
    cursor.execute(query,(time_start,time_end))
    
    for (my_ip,next_ip,next_port,prev_circ_id,next_circ_id,direction,stream_id,is_origin) in cursor:
        t=link(my_ip,next_ip,next_port,prev_circ_id,next_circ_id,direction,stream_id,is_origin)
        if(is_origin):
            origin[t.my_ip][t.prev_circ_id]=t
        else:
            relay[t.my_ip][t.prev_circ_id]=t
    #as test

#     a=link("12","34",1,"10.0.0.1","10.0.0.3",600,123,True)
#     b=link("34","56",2,"10.0.0.3","10.0.0.4",700,123)
#     c=link("56","11111",3,"10.0.0.4","10.0.0.7",123,999)
#     origin[a.my_ip]={a.prev_circ_id:a}
#     relay[b.my_ip]={b.prev_circ_id:b}
#     relay[c.my_ip]={c.prev_circ_id:c}
    cnx.close()
    return origin,relay

origins,relays=defaultdict(dict),defaultdict(dict) #global var
#     return None
def searchNext(p,relays):
    #todo using direction to estimate whether this relay is right
    # p:type link  --the previous one 
    # relay:type dict {"my_ip":{"prev_circ_id":link}} --the relay dict
    # return list()  --the list from p to end looks like [p1,[p2,[p3,None]]]
    if(p.next_ip in relays):
        nls=relays[p.next_ip]
        print(nls)
        if(p.next_circ_id in nls): # prev_relay.next_circ_id equals to the next one's prev_circ_id
            t=searchNext(nls[p.next_circ_id],relays)
            return [p,t]
    return None

def printFixLinks(fixlinks):
    # type fixlinks:dict("origin_ip":list [[p1,[p2,None]],[p1,[p3,[p4,None]]]])
    for i in fixlinks:
        print(i)
        print(a[i])
        for ii in a[i]:
            while(ii!=None):
                print(ii[0].next_ip)
                ii=ii[1]

def linkAll():
    # rtype fixlinks: dict("origin_ip":list [[p1,[p2,None]],[p1,[p3,[p4,None]]]])
    fixlinks=defaultdict(list)
    origins,relays=readFromDB()
    print(origins,relays)
    for oStr in origins:# though all the origin #this is a p
        o=origins[oStr]
        for prev_circ_id in o: #all the next ip
            p=o[prev_circ_id]# one origin link point
            print(p.next_ip)
            t=searchNext(p,relays)
            if(t!=None):
#                 using defaultdict(list)
                #if(oStr not in fixlinks):
                  #  fixlinks[oStr]=list()
                fixlinks[oStr].append(t)
    return fixlinks
#             if(next_ip not in relays): #not such relay
#                 continue
#             else:
#                 p=relay[next_ip]
#                 searchNext(p)
if __name__=="__main__":
    a=linkAll()
    printFixLinks(a)

IndentationError: expected an indented block (<ipython-input-3-3d926412317a>, line 48)